# YOLO Video Detection con Supervision

## ¿Qué es YOLO?

**YOLO** (You Only Look Once) es una familia de modelos de deep learning para **detección de objetos en tiempo real**.

A diferencia de otros detectores que analizan la imagen en múltiples pasadas, YOLO procesa toda la imagen en una sola pasada de la red neuronal, lo que lo hace extremadamente rápido.

### Versiones de YOLO

| Versión | Año | Características |
|---------|-----|----------------|
| YOLOv8 | 2023 | Arquitectura mejorada, fácil de usar |
| YOLOv9 | 2024 | Programmable Gradient Information (PGI) |
| YOLOv10 | 2024 | Entrenamiento sin NMS (Non-Maximum Suppression) |
| YOLO11 | 2024 | Última versión, mejor balance velocidad/precisión |

## ¿Qué es Supervision?

**Supervision** es una librería de Roboflow que facilita:
- Convertir resultados de diferentes modelos a un formato unificado
- Tracking de objetos (seguimiento entre frames)
- Anotaciones visuales (dibujar boxes, etiquetas, trazas)
- Procesamiento de video

## Objetivo del Notebook

Aprenderemos a:
1. Cargar un modelo YOLO y detectar objetos
2. Usar tracking para seguir objetos entre frames
3. Visualizar resultados con anotaciones
4. Procesar videos completos y webcam en tiempo real

---
## 1. Instalación e Imports

### Librerías necesarias

| Librería | Propósito |
|----------|----------|
| `ultralytics` | Modelos YOLO oficiales |
| `supervision` | Tracking y anotaciones |
| `opencv-python` | Manipulación de imágenes/video |
| `numpy` | Operaciones con arrays |
| `matplotlib` | Visualización en notebooks |

In [ ]:
# ============================================================
# INSTALACIÓN (ejecutar solo la primera vez)
# ============================================================
# Descomenta la siguiente línea si no tienes las librerías:
# !pip install ultralytics supervision opencv-python

In [ ]:
# ============================================================
# IMPORTS
# ============================================================

# OpenCV: lectura/escritura de imágenes y video
import cv2

# NumPy: los frames son arrays de numpy (altura x anchura x 3 canales BGR)
import numpy as np

# Supervision: tracking, anotaciones, procesamiento de video
import supervision as sv

# Ultralytics: modelos YOLO
from ultralytics import YOLO

# Matplotlib: visualización en el notebook
import matplotlib.pyplot as plt

# Verificar versiones
print(f"✓ Supervision version: {sv.__version__}")
print(f"✓ OpenCV version: {cv2.__version__}")

---
## 2. Cargar el Modelo YOLO

### ¿Cómo funciona YOLO internamente?

```
Imagen (640x640 RGB)
        │
        ▼
┌───────────────────┐
│   Backbone CNN    │  ← Extrae características (ResNet, CSPNet, etc.)
│   (Feature maps)  │
└───────────────────┘
        │
        ▼
┌───────────────────┐
│      Neck         │  ← Combina características de diferentes escalas (FPN, PANet)
│  (Multi-scale)    │
└───────────────────┘
        │
        ▼
┌───────────────────┐
│      Head         │  ← Predice boxes, clases y confianza
│  (Detection)      │
└───────────────────┘
        │
        ▼
Lista de detecciones: [(x1,y1,x2,y2, clase, confianza), ...]
```

### Tamaños de modelo disponibles

| Modelo | Parámetros | Velocidad | Precisión (mAP) | Uso recomendado |
|--------|------------|-----------|-----------------|------------------|
| `yolo11n.pt` | 2.6M | ⚡⚡⚡⚡⚡ | 39.5% | Tiempo real, edge devices |
| `yolo11s.pt` | 9.4M | ⚡⚡⚡⚡ | 47.0% | Balance velocidad/precisión |
| `yolo11m.pt` | 20.1M | ⚡⚡⚡ | 51.5% | Uso general |
| `yolo11l.pt` | 25.3M | ⚡⚡ | 53.4% | Alta precisión |
| `yolo11x.pt` | 56.9M | ⚡ | 54.7% | Máxima precisión |

**mAP** = Mean Average Precision (métrica estándar de detección, 0-100%)

In [ ]:
# ============================================================
# CARGAR MODELO YOLO
# ============================================================
# La primera vez se descarga automáticamente (~5-50 MB según el modelo)
# Los pesos se guardan en el directorio actual

MODEL_NAME = "yolo11n.pt"  # Usamos nano para velocidad

print(f"Cargando modelo {MODEL_NAME}...")
model = YOLO(MODEL_NAME)
print(f"✓ Modelo cargado correctamente")

In [ ]:
# ============================================================
# EXPLORAR LAS CLASES DEL MODELO
# ============================================================
# YOLO viene pre-entrenado en COCO dataset (80 clases comunes)

print(f"El modelo puede detectar {len(model.names)} clases diferentes:\n")

# Mostrar todas las clases en columnas
clases = list(model.names.values())
for i in range(0, len(clases), 4):
    row = clases[i:i+4]
    print("  ".join(f"{i+j:2d}. {c:<15}" for j, c in enumerate(row)))

---
## 3. Detección en una Imagen

### El flujo de detección

```
┌──────────────┐      ┌──────────────┐      ┌──────────────────────┐
│    Imagen    │  ──► │  model(img)  │  ──► │  Results object      │
│  (np.array)  │      │              │      │  - boxes (xyxy)      │
│  BGR format  │      │              │      │  - confidence scores │
│              │      │              │      │  - class IDs         │
└──────────────┘      └──────────────┘      └──────────────────────┘
```

### Formato de Bounding Box: xyxy

```
    x1 (left)
    │
    ▼
    ┌─────────────────┐ ◄── y1 (top)
    │                 │
    │     OBJETO      │
    │                 │
    └─────────────────┘ ◄── y2 (bottom)
                      ▲
                      │
                   x2 (right)

xyxy = [x1, y1, x2, y2] = [left, top, right, bottom]
```

In [ ]:
# ============================================================
# CAPTURAR IMAGEN DE LA WEBCAM
# ============================================================
# cv2.VideoCapture(0) abre la webcam por defecto
# También podrías cargar una imagen: frame = cv2.imread("imagen.jpg")

cap = cv2.VideoCapture(0)  # 0 = webcam por defecto
ret, frame = cap.read()     # ret = True si se capturó correctamente
cap.release()               # Liberar la webcam

if ret:
    print(f"✓ Imagen capturada: {frame.shape}")
    print(f"  - Altura: {frame.shape[0]} píxeles")
    print(f"  - Anchura: {frame.shape[1]} píxeles") 
    print(f"  - Canales: {frame.shape[2]} (BGR)")
    
    # Mostrar imagen (convertir BGR->RGB para matplotlib)
    plt.figure(figsize=(10, 6))
    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    plt.title("Imagen original de la webcam")
    plt.axis('off')
    plt.show()
else:
    print("⚠️ No se pudo capturar imagen de la webcam")
    print("   Creando imagen de prueba...")
    frame = np.zeros((480, 640, 3), dtype=np.uint8)

### Ejecutar la detección

El parámetro `conf` es el **umbral de confianza**:
- `conf=0.5` → Solo detecciones con >50% de confianza
- Valor alto (0.7-0.9) → Menos detecciones, más precisas
- Valor bajo (0.1-0.3) → Más detecciones, incluye falsos positivos

In [ ]:
# ============================================================
# EJECUTAR DETECCIÓN YOLO
# ============================================================

CONFIDENCE_THRESHOLD = 0.5  # 50% mínimo de confianza

# model() devuelve una lista de Results (uno por imagen)
# Como pasamos una sola imagen, tomamos el primer elemento [0]
results = model(frame, conf=CONFIDENCE_THRESHOLD, verbose=False)[0]

print(f"Inferencia completada")
print(f"  - Objetos detectados: {len(results.boxes)}")

### Convertir a formato Supervision

**¿Por qué usar `sv.Detections`?**

Supervision proporciona una clase unificada que funciona con cualquier detector:
- `sv.Detections.from_ultralytics()` - Para YOLO
- `sv.Detections.from_detectron2()` - Para Detectron2
- `sv.Detections.from_transformers()` - Para HuggingFace

Esto permite usar el mismo código de anotación/tracking independientemente del modelo.

In [ ]:
# ============================================================
# CONVERTIR A FORMATO SUPERVISION
# ============================================================

detections = sv.Detections.from_ultralytics(results)

# Explorar la estructura de Detections
print("Estructura de sv.Detections:")
print(f"  - xyxy: array de bounding boxes [{len(detections)}x4]")
print(f"  - confidence: array de confianzas [{len(detections)}]")
print(f"  - class_id: array de IDs de clase [{len(detections)}]")
print(f"  - tracker_id: {detections.tracker_id} (None hasta aplicar tracking)")

print(f"\n{'='*60}")
print(f"DETECCIONES ENCONTRADAS: {len(detections)}")
print(f"{'='*60}")

for i, (xyxy, conf, class_id) in enumerate(zip(
    detections.xyxy, 
    detections.confidence, 
    detections.class_id
)):
    class_name = model.names[class_id]
    x1, y1, x2, y2 = xyxy.astype(int)
    width = x2 - x1
    height = y2 - y1
    
    print(f"\n{i+1}. {class_name.upper()}")
    print(f"   Confianza: {conf:.1%}")
    print(f"   Bounding box: ({x1}, {y1}) → ({x2}, {y2})")
    print(f"   Tamaño: {width}x{height} píxeles")

### Visualizar con Annotators

**Annotators disponibles en Supervision:**

| Annotator | Descripción |
|-----------|-------------|
| `BoxAnnotator` | Dibuja rectángulos alrededor de objetos |
| `LabelAnnotator` | Añade texto (clase, confianza, ID) |
| `MaskAnnotator` | Rellena el área del objeto (segmentación) |
| `TraceAnnotator` | Dibuja la trayectoria histórica |
| `HeatMapAnnotator` | Mapa de calor de actividad |
| `CircleAnnotator` | Círculos en lugar de boxes |

In [ ]:
# ============================================================
# CREAR ANOTADORES
# ============================================================

# BoxAnnotator: dibuja los rectángulos
# - thickness: grosor de la línea en píxeles
box_annotator = sv.BoxAnnotator(
    thickness=2  # Grosor del borde del rectángulo
)

# LabelAnnotator: dibuja las etiquetas de texto
# - text_scale: tamaño del texto (0.5 = mitad del default)
# - text_thickness: grosor del texto
label_annotator = sv.LabelAnnotator(
    text_scale=0.5,
    text_thickness=1
)

In [ ]:
# ============================================================
# GENERAR ETIQUETAS PERSONALIZADAS
# ============================================================
# Creamos una lista de strings para cada detección
# Formato: "clase confianza" (ej: "person 0.95")

labels = []
for class_id, confidence in zip(detections.class_id, detections.confidence):
    class_name = model.names[class_id]  # Convertir ID a nombre
    label = f"{class_name} {confidence:.2f}"  # ej: "person 0.95"
    labels.append(label)

print(f"Etiquetas generadas: {labels}")

In [ ]:
# ============================================================
# ANOTAR LA IMAGEN
# ============================================================
# IMPORTANTE: Siempre trabajar sobre una COPIA del frame
# para no modificar el original

# Paso 1: Copiar el frame original
annotated = frame.copy()

# Paso 2: Dibujar bounding boxes
annotated = box_annotator.annotate(
    scene=annotated,           # Imagen sobre la que dibujar
    detections=detections      # Detecciones a dibujar
)

# Paso 3: Dibujar etiquetas
annotated = label_annotator.annotate(
    scene=annotated,
    detections=detections,
    labels=labels              # Lista de strings personalizados
)

# Visualizar resultado
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
plt.title(f"Detección YOLO - {len(detections)} objetos detectados")
plt.axis('off')
plt.show()

---
## 4. Tracking de Objetos con ByteTrack

### ¿Qué es el tracking?

**Detección** identifica objetos en cada frame de forma independiente.

**Tracking** asocia las detecciones entre frames para saber que "el objeto en el frame 1" es "el mismo objeto en el frame 2".

```
Frame 1          Frame 2          Frame 3
┌─────────┐      ┌─────────┐      ┌─────────┐
│ ┌───┐   │      │   ┌───┐ │      │     ┌───┐│
│ │ A │   │  ──► │   │ ? │ │  ──► │     │ ? ││
│ └───┘   │      │   └───┘ │      │     └───┘│
└─────────┘      └─────────┘      └─────────┘

Sin tracking: A, B, C (diferentes IDs cada frame)
Con tracking: A, A, A (mismo ID = mismo objeto)
```

### ¿Cómo funciona ByteTrack?

1. **Asociación por posición**: Si un objeto está cerca de donde estaba antes, probablemente es el mismo
2. **Predicción de movimiento**: Usa filtro de Kalman para predecir dónde estará el objeto
3. **Manejo de oclusiones**: Mantiene IDs aunque el objeto desaparezca temporalmente

In [ ]:
# ============================================================
# CREAR TRACKER
# ============================================================
# ByteTrack es el algoritmo de tracking incluido en supervision
# Mantiene un estado interno de los objetos trackeados

tracker = sv.ByteTrack(
    # Parámetros opcionales:
    # track_activation_threshold=0.25,  # Confianza mínima para nuevo track
    # lost_track_buffer=30,             # Frames antes de eliminar track perdido
    # minimum_matching_threshold=0.8,   # Umbral para asociar detecciones
    # frame_rate=30                     # FPS del video
)

print("✓ Tracker ByteTrack creado")

In [ ]:
# ============================================================
# APLICAR TRACKING A LAS DETECCIONES
# ============================================================
# update_with_detections() hace dos cosas:
# 1. Asocia detecciones actuales con tracks existentes
# 2. Añade el campo tracker_id a las detecciones

detections_with_tracking = tracker.update_with_detections(detections)

print("Detecciones CON tracking:")
print(f"  - tracker_id: {detections_with_tracking.tracker_id}")

if detections_with_tracking.tracker_id is not None:
    print(f"\nObjetos trackeados:")
    for tid, cid, conf in zip(
        detections_with_tracking.tracker_id,
        detections_with_tracking.class_id,
        detections_with_tracking.confidence
    ):
        print(f"  ID #{tid}: {model.names[cid]} ({conf:.1%})")
else:
    print("  (No hay objetos trackeados)")

---
## 5. Procesar Video Completo

### Flujo de procesamiento de video

```
┌─────────────┐     ┌─────────────────┐     ┌─────────────┐
│   Video     │     │    Para cada    │     │   Video     │
│   entrada   │ ──► │     frame:      │ ──► │   salida    │
│  (MP4/AVI)  │     │   callback()    │     │  (MP4/AVI)  │
└─────────────┘     └─────────────────┘     └─────────────┘
                           │
                           ▼
                    ┌─────────────┐
                    │  callback:  │
                    │ 1. Detectar │
                    │ 2. Trackear │
                    │ 3. Anotar   │
                    │ 4. Return   │
                    └─────────────┘
```

### sv.process_video()

Esta función de supervision:
1. Abre el video de entrada
2. Lee cada frame
3. Llama a tu función `callback(frame, index)`
4. Escribe el frame retornado al video de salida
5. Cierra los archivos automáticamente

In [ ]:
# ============================================================
# CONFIGURACIÓN DEL PROCESAMIENTO DE VIDEO
# ============================================================

VIDEO_INPUT = "test_video.mp4"     # Video de entrada (cambiar por tu video)
VIDEO_OUTPUT = "output_notebook.mp4"  # Video de salida
CONFIDENCE = 0.5                   # Umbral de confianza

# IMPORTANTE: Reiniciar el tracker para cada video nuevo
# Si no, los IDs continuarían del video anterior
tracker = sv.ByteTrack()

# Crear los tres anotadores que usaremos
box_annotator = sv.BoxAnnotator(thickness=2)
label_annotator = sv.LabelAnnotator(text_scale=0.5, text_thickness=1)
trace_annotator = sv.TraceAnnotator(
    thickness=2,      # Grosor de la línea de traza
    trace_length=50   # Cuántos frames de historia mostrar
)

print("✓ Configuración lista")

In [ ]:
# ============================================================
# DEFINIR LA FUNCIÓN CALLBACK
# ============================================================
# Esta función se ejecutará para CADA frame del video
# Debe recibir (frame, frame_idx) y devolver el frame anotado

def process_frame(frame: np.ndarray, frame_idx: int) -> np.ndarray:
    """
    Procesa un frame individual del video.
    
    Args:
        frame: Imagen BGR del frame actual (numpy array)
        frame_idx: Índice del frame (0, 1, 2, ...)
    
    Returns:
        Frame anotado con detecciones, tracking y trazas
    """
    # ──────────────────────────────────────────────────────
    # PASO 1: DETECCIÓN YOLO
    # ──────────────────────────────────────────────────────
    # verbose=False evita que imprima info en cada frame
    results = model(frame, conf=CONFIDENCE, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(results)
    
    # ──────────────────────────────────────────────────────
    # PASO 2: TRACKING
    # ──────────────────────────────────────────────────────
    # Asocia detecciones con IDs persistentes
    detections = tracker.update_with_detections(detections)
    
    # ──────────────────────────────────────────────────────
    # PASO 3: GENERAR ETIQUETAS
    # ──────────────────────────────────────────────────────
    # Formato: "#ID clase confianza" (ej: "#1 person 0.95")
    if detections.tracker_id is not None:
        labels = [
            f"#{tid} {model.names[cid]} {conf:.2f}"
            for tid, cid, conf in zip(
                detections.tracker_id,
                detections.class_id,
                detections.confidence
            )
        ]
    else:
        labels = [
            f"{model.names[cid]} {conf:.2f}"
            for cid, conf in zip(detections.class_id, detections.confidence)
        ]
    
    # ──────────────────────────────────────────────────────
    # PASO 4: ANOTAR FRAME
    # ──────────────────────────────────────────────────────
    # Orden importante: primero traces (fondo), luego boxes, luego labels
    annotated = frame.copy()
    annotated = trace_annotator.annotate(annotated, detections=detections)  # Trazas
    annotated = box_annotator.annotate(annotated, detections=detections)    # Boxes
    annotated = label_annotator.annotate(annotated, detections=detections, labels=labels)  # Labels
    
    # Mostrar progreso cada 50 frames
    if frame_idx % 50 == 0:
        print(f"  Frame {frame_idx}: {len(detections)} objetos detectados")
    
    return annotated

print("✓ Función callback definida")

In [ ]:
# ============================================================
# VERIFICAR VIDEO DE ENTRADA
# ============================================================
import os

if os.path.exists(VIDEO_INPUT):
    # sv.VideoInfo extrae metadatos del video
    video_info = sv.VideoInfo.from_video_path(VIDEO_INPUT)
    
    print(f"📹 Video de entrada: {VIDEO_INPUT}")
    print(f"   ├── Resolución: {video_info.width} x {video_info.height}")
    print(f"   ├── FPS: {video_info.fps}")
    print(f"   ├── Total frames: {video_info.total_frames}")
    print(f"   └── Duración: {video_info.total_frames / video_info.fps:.1f} segundos")
else:
    print(f"⚠️ No se encontró el video: {VIDEO_INPUT}")
    print(f"   Opciones:")
    print(f"   1. Descarga un video de prueba")
    print(f"   2. Cambia VIDEO_INPUT a la ruta de tu video")

In [ ]:
# ============================================================
# PROCESAR EL VIDEO
# ============================================================
# sv.process_video() se encarga de:
# - Abrir el video de entrada
# - Leer frame por frame
# - Llamar a callback(frame, idx) para cada frame
# - Escribir el resultado al video de salida
# - Cerrar los archivos

if os.path.exists(VIDEO_INPUT):
    print(f"\n🔄 Procesando video...\n")
    
    sv.process_video(
        source_path=VIDEO_INPUT,   # Video original
        target_path=VIDEO_OUTPUT,  # Video con anotaciones
        callback=process_frame     # Nuestra función
    )
    
    print(f"\n✅ Video guardado en: {VIDEO_OUTPUT}")
    print(f"   Tamaño: {os.path.getsize(VIDEO_OUTPUT) / 1024 / 1024:.1f} MB")

---
## 6. Detección en Tiempo Real (Webcam)

### Diferencia con procesamiento de archivo

| Aspecto | Archivo | Webcam |
|---------|---------|--------|
| Fuente | `sv.process_video()` | `cv2.VideoCapture(0)` |
| Velocidad | Procesa lo más rápido posible | Limitado por FPS de la cámara |
| Interactividad | No | Sí (presionar 'q' para salir) |
| Visualización | Guarda a archivo | `cv2.imshow()` en ventana |

### Flujo de webcam

```python
while True:
    frame = webcam.read()      # Capturar frame
    annotated = process(frame) # Detectar + anotar
    cv2.imshow(annotated)      # Mostrar en ventana
    if key == 'q': break       # Salir si presiona 'q'
```

In [ ]:
# ============================================================
# FUNCIÓN DE DETECCIÓN EN WEBCAM
# ============================================================

def run_webcam_detection(duration_seconds: int = 30):
    """
    Ejecuta detección YOLO en tiempo real usando la webcam.
    
    Args:
        duration_seconds: Duración máxima en segundos (default: 30)
    
    Controles:
        - Presiona 'q' para salir antes de tiempo
    """
    # ──────────────────────────────────────────────────────
    # ABRIR WEBCAM
    # ──────────────────────────────────────────────────────
    cap = cv2.VideoCapture(0)  # 0 = webcam por defecto
    
    if not cap.isOpened():
        print("❌ Error: No se puede abrir la webcam")
        return
    
    # Obtener FPS de la webcam (típicamente 30)
    fps = cap.get(cv2.CAP_PROP_FPS) or 30
    max_frames = int(duration_seconds * fps)
    
    # ──────────────────────────────────────────────────────
    # INICIALIZAR TRACKER Y ANOTADORES
    # ──────────────────────────────────────────────────────
    tracker = sv.ByteTrack()  # Nuevo tracker para esta sesión
    box_annotator = sv.BoxAnnotator(thickness=2)
    label_annotator = sv.LabelAnnotator(text_scale=0.5, text_thickness=1)
    trace_annotator = sv.TraceAnnotator(thickness=2, trace_length=50)
    
    print(f"🎥 Iniciando detección en webcam")
    print(f"   Duración máxima: {duration_seconds}s")
    print(f"   Presiona 'q' en la ventana para salir\n")
    
    frame_count = 0
    
    # ──────────────────────────────────────────────────────
    # BUCLE PRINCIPAL
    # ──────────────────────────────────────────────────────
    while frame_count < max_frames:
        # Capturar frame
        ret, frame = cap.read()
        if not ret:
            print("⚠️ Error leyendo frame")
            break
        
        # Detección
        results = model(frame, conf=0.5, verbose=False)[0]
        detections = sv.Detections.from_ultralytics(results)
        
        # Tracking
        detections = tracker.update_with_detections(detections)
        
        # Etiquetas con ID de tracking
        if detections.tracker_id is not None:
            labels = [
                f"#{tid} {model.names[cid]}"
                for tid, cid in zip(detections.tracker_id, detections.class_id)
            ]
        else:
            labels = [model.names[cid] for cid in detections.class_id]
        
        # Anotar
        annotated = frame.copy()
        annotated = trace_annotator.annotate(annotated, detections)
        annotated = box_annotator.annotate(annotated, detections)
        annotated = label_annotator.annotate(annotated, detections, labels=labels)
        
        # Mostrar en ventana
        cv2.imshow("YOLO Webcam - Presiona 'q' para salir", annotated)
        
        # Verificar si se presionó 'q'
        # cv2.waitKey(1) espera 1ms y devuelve la tecla presionada
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("\n👋 Saliendo por petición del usuario")
            break
        
        frame_count += 1
    
    # ──────────────────────────────────────────────────────
    # LIMPIAR RECURSOS
    # ──────────────────────────────────────────────────────
    cap.release()              # Liberar webcam
    cv2.destroyAllWindows()    # Cerrar ventanas
    
    print(f"\n✅ Detección finalizada")
    print(f"   Frames procesados: {frame_count}")
    print(f"   Duración: {frame_count/fps:.1f}s")

In [ ]:
# ============================================================
# EJECUTAR DETECCIÓN EN WEBCAM
# ============================================================
# Descomenta la siguiente línea para ejecutar (30 segundos)

# run_webcam_detection(30)

---
## 7. Resumen y Arquitectura

### Pipeline Completo

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                           PIPELINE DE DETECCIÓN                             │
└─────────────────────────────────────────────────────────────────────────────┘

  ┌─────────────┐      ┌─────────────────┐      ┌─────────────┐      ┌─────────────┐
  │   ENTRADA   │      │      YOLO       │      │  BYTETRACK  │      │ ANNOTATORS  │
  │             │      │                 │      │             │      │             │
  │ • Webcam    │ ───► │ • Backbone CNN  │ ───► │ • Kalman    │ ───► │ • Box       │
  │ • Video     │      │ • Neck FPN      │      │   Filter    │      │ • Label     │
  │ • Imagen    │      │ • Head          │      │ • Hungarian │      │ • Trace     │
  │             │      │                 │      │   Matching  │      │             │
  └─────────────┘      └─────────────────┘      └─────────────┘      └─────────────┘
        │                      │                      │                    │
        ▼                      ▼                      ▼                    ▼
  ┌─────────────┐      ┌─────────────────┐      ┌─────────────┐      ┌─────────────┐
  │  np.array   │      │  sv.Detections  │      │ + tracker_  │      │   Frame     │
  │  BGR        │      │  • xyxy         │      │   id        │      │  anotado    │
  │  HxWx3      │      │  • confidence   │      │             │      │             │
  │             │      │  • class_id     │      │             │      │             │
  └─────────────┘      └─────────────────┘      └─────────────┘      └─────────────┘
```

### Clases y Funciones Principales

| Componente | Clase/Función | Descripción |
|------------|---------------|-------------|
| **Modelo** | `YOLO("yolo11n.pt")` | Carga modelo pre-entrenado |
| **Inferencia** | `model(frame)` | Ejecuta detección |
| **Detecciones** | `sv.Detections.from_ultralytics()` | Convierte a formato unificado |
| **Tracking** | `sv.ByteTrack()` | Crea tracker |
| **Tracking** | `tracker.update_with_detections()` | Asigna IDs persistentes |
| **Anotación** | `sv.BoxAnnotator()` | Dibuja rectángulos |
| **Anotación** | `sv.LabelAnnotator()` | Dibuja etiquetas |
| **Anotación** | `sv.TraceAnnotator()` | Dibuja trayectorias |
| **Video** | `sv.process_video()` | Procesa video con callback |
| **Video** | `sv.VideoInfo.from_video_path()` | Obtiene metadatos |

### Atributos de sv.Detections

| Atributo | Tipo | Descripción |
|----------|------|-------------|
| `xyxy` | `np.array [N, 4]` | Bounding boxes [x1, y1, x2, y2] |
| `confidence` | `np.array [N]` | Confianza de cada detección (0-1) |
| `class_id` | `np.array [N]` | ID de clase (0-79 para COCO) |
| `tracker_id` | `np.array [N]` | ID de tracking (None si no hay tracking) |
| `mask` | `np.array [N, H, W]` | Máscaras de segmentación (opcional) |
| `data` | `dict` | Datos adicionales personalizados |

---
## 8. Ejercicios Propuestos

### Ejercicio 1: Filtrar por clase
Modifica `process_frame()` para detectar solo personas:
```python
# Filtrar detecciones donde class_id == 0 (person en COCO)
mask = detections.class_id == 0
detections = detections[mask]
```

### Ejercicio 2: Contar objetos
Añade un contador de objetos únicos usando los tracker_id:
```python
unique_ids = set()
# En cada frame: unique_ids.update(detections.tracker_id)
# Al final: print(f"Objetos únicos: {len(unique_ids)}")
```

### Ejercicio 3: Zona de conteo
Cuenta cuántos objetos cruzan una línea usando `sv.LineZone`:
```python
line = sv.LineZone(start=(0, 300), end=(640, 300))
line.trigger(detections)  # Devuelve (in_count, out_count)
```

### Ejercicio 4: Guardar detecciones
Exporta las detecciones a un archivo CSV o JSON para análisis posterior.